In [46]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, timezone, date
from decouple import config
import string
from copy import copy

In [47]:
# Credentials Data lake
dluname = config('DBUNAME')
dlpwd = config('DBPWD')
dlhost = config('HOST')
dldbname = config('DBNAME')
port = config('PORT')

# Credentials Datawarehouse
dwhuname = config('DWHUNAME')
dwhpwd = config('DWHPWD')
dwhhost = config('DWHHOST')
dwhdbname = config('DWHDBNAME')

In [78]:
def fetch_load_data(cur, asset, method):
    
    #define variables for fetching data
    table_name_str = "'table_yf_"+asset+"'"
    table_name = 'table_yf_'+asset
    
    df = pd.DataFrame()
    
 
    while True:
        #column names of the table in the data lake
        try:
            cur.execute("select COLUMN_NAME from information_schema.columns where table_name="+table_name_str)
            column_names = [row[0] for row in cur]
        
            col_ind_date = column_names.index('date')
            col_ind_open = column_names.index('Open')
            col_ind_high = column_names.index('High')
            col_ind_low = column_names.index('Low')
            col_ind_close = column_names.index('Close')
            col_ind_vol = column_names.index('Volume')
            col_ind_asset = column_names.index('Ins_label')      
        except Exception as e: 
            print("Column indexes could no be defined")
            print(e)
            break  
        

        
        #fetch data in data lake and build dataframe
        try:            
            cur.execute("SELECT * FROM "+table_name+";")
            result = cur.fetchall()
                
            for row in result:
                df = df.append(pd.DataFrame([{'date': row[col_ind_date], 
                                    'asset': row[col_ind_asset],
                                     'open': row[col_ind_open],
                                     'high': row[col_ind_high],
                                     'low': row[col_ind_low],
                                     'close': row[col_ind_close],
                                     'volume': row[col_ind_vol]}]))                  
        except psycopg2.Error as e:
            print("Error: select *")
            print(e)

        df = df.reset_index(drop=True)   
        
        #initialize dataframe for the data load
        df2 = pd.DataFrame()
        
        #define variable for different timedeltas
        td1 = pd.Timedelta(1, "d")
        td2 = pd.Timedelta(2, "d")
        td3 = pd.Timedelta(3, "d")
        td4 = pd.Timedelta(4, "d")
        td5 = pd.Timedelta(5, "d")
        
        #building dataframe for the data load        
        for index, row in df.iterrows():
            if index == 0:
                df2 = df2.append(row)
            else:
                #take last row in the df2
                last_row = df2.iloc[-1:]
                
                #calculate timedelta between actual row and last row in df2               
                act_date =  pd.to_datetime(row['date'])
                last_date = last_row['date']
                timestamp = ((last_date - np.datetime64('1970-01-01T00:00:00')) / np.timedelta64(1, 's'))
                last_date = datetime.utcfromtimestamp(timestamp)
                diff = act_date - last_date
                
                #depending on timedelta different actions are needed to fill dates without data
                #timedelta = 1 day: append actual row to df2
                if diff == td1:        
                    df2 = df2.append(row)
                #timedelta = 2 days: One date has no data. Copy last row of df2, change the date and append this row and the
                # actual row to df2
                elif diff == td2:
                    #Copy last row of df2
                    new_row = copy(last_row)
                    #Calculate the date which has missing data
                    new_date1 = last_date + timedelta(1)
                    #Change the date in the row of df2, which has been copied
                    new_row1['date'] = new_date1
                    #Append the row on the date which has missing data to df2
                    df2 = df2.append(new_row)
                    #append actual row to df2
                    df2 = df2.append(row)
                #timedelta = 3 days: Two dates has no data. Same procedure as for timedelta = 2, but one row more has to append                 
                elif diff == td3:
                    new_row = copy(last_row)
                    new_date1 = last_date + timedelta(1)
                    new_date2 = last_date + timedelta(2)
            
                    new_row['date'] = new_date1
                    df2 = df2.append(new_row)            
                    new_row['date'] = new_date2
                    df2 = df2.append(new_row)
                    df2 = df2.append(row)
                #timedelta = 4 days: Two dates has no data. Same procedure as for timedelta = 2, but two row more has to append                    
                elif diff == td4:
                    new_row = copy(last_row)
                    new_date1 = last_date + timedelta(1)
                    new_date2 = last_date + timedelta(2)
                    new_date3 = last_date + timedelta(3)
            
                    new_row['date'] = new_date1
                    df2 = df2.append(new_row)            
                    new_row['date'] = new_date2
                    df2 = df2.append(new_row)            
                    new_row['date'] = new_date3
                    df2 = df2.append(new_row)
                    df2 = df2.append(row)
                #timedelta = 5 days: Two dates has no data. Same procedure as for timedelta = 2, but three row more has to append
                elif diff == td5:
                    new_row = copy(last_row)
                    new_date1 = last_date + timedelta(1)
                    new_date2 = last_date + timedelta(2)
                    new_date3 = last_date + timedelta(3)
                    new_date4 = last_date + timedelta(4)           
            
                    new_row['date'] = new_date1
                    df2 = df2.append(new_row)            
                    new_row['date'] = new_date2
                    df2 = df2.append(new_row)            
                    new_row['date'] = new_date3
                    df2 = df2.append(new_row)
                    new_row['date'] = new_date4
                    df2 = df2.append(new_row)        
                    df2 = df2.append(row)
                else:
                    print("No action for this timedelta defined")
        
        #reset index of df2
        df2 = df2.reset_index(drop=True)    
        
        #define loading variables 
        dwhtblname = 'finance'
        if_ex_val = method
    
        #load data
        try:
            conn_string = 'postgresql://'+dwhuname+':'+dwhpwd+'@'+dwhhost+':'+port+'/'+dwhdbname
            engine = create_engine(conn_string)
            df2.to_sql(dwhtblname, conn_string, if_exists = if_ex_val, index=False)
            print(table_name+" loaded")
        except Exception as e:
            print(e)
            print("Data load failed: " + table_name)
            break
        break    
    

In [53]:
def close_conn_to_dl(cur, conn):
    try:
        cur.close()
        conn.close()
        print("connection closed")
    except psycopg2.Error as e: 
        print("Error: Could not close")
        print(e)    

In [54]:
def conn_to_dl():
    try: 
        conn = psycopg2.connect("host=" + dlhost + " dbname=" + dldbname + " user=" + dluname + " password="+dlpwd)
    except psycopg2.Error as e: 
        print("Error: Could not make connection to the Postgres database")
        print(e)
    
    try: 
        cur = conn.cursor()
    except psycopg2.Error as e: 
        print("Error: Could not get curser to the Database")
        print(e)
    
    # Auto commit
    conn.set_session(autocommit=True)
    print("connected")
    return cur, conn

In [72]:
assets={'gold','silver','msci_world','euro_stoxx','smi','nasdaq'}
#assets={'gold'}

In [79]:
def main():
    
    #open connection
    cur, conn = conn_to_dl()
    
    # process data
    for asset in assets:
        fetch_load_data(cur, asset,'replace')  
    
    #close connection
    close_conn_to_dl(cur, conn)    

In [80]:
if __name__ == "__main__":
    main()

connected


C:\Users\C\AppData\Local\Temp/ipykernel_20260/1688183079.py:71: DeprecationWarning: an integer is required (got type Series).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  last_date = datetime.utcfromtimestamp(timestamp)


table_yf_smi loaded
table_yf_euro_stoxx loaded
table_yf_msci_world loaded
table_yf_silver loaded
table_yf_nasdaq loaded
table_yf_gold loaded
connection closed
